In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#PATH = '/home/adriana/Projects/CNV/230404-fold80-investigation'
PATH = '/Users/adriana/Projects/cnv/gCNV-validation'

In [ ]:
run50HS = pd.read_csv(f'{PATH}/0050-hs-metrics.txt', sep='\t')
run42HS = pd.read_csv(f'{PATH}/0042-hs-metrics.txt', sep='\t')
run39HS = pd.read_csv(f'{PATH}/0039-hs-metrics.txt', sep='\t')
run50HS['Run'] = '0050'
run42HS['Run'] = '0042'
run39HS['Run'] = '0039'

run50FQ = pd.read_csv(f'{PATH}/0050-fastqc.txt', sep='\t')
run39FQ = pd.read_csv(f'{PATH}/0039-fastqc.txt', sep='\t')
run42FQ = pd.read_csv(f'{PATH}/0042-fastqc.txt', sep='\t')
run39FQ['Run'] = '0039'
run42FQ['Run'] = '0042'
run50FQ['Run'] = '0050'

In [ ]:
allrunsHS = pd.concat([run50HS, run42HS, run39HS])
plt.style.use('ggplot')

In [ ]:
sns.scatterplot(x='Sample', y='FOLD_80_BASE_PENALTY', hue='Run', data=allrunsHS)
plt.axhline(y=1.3,color='r',linestyle='--')
#plt.xticks(rotation=90)

In [ ]:
CNVsamples=allrunsHS.loc[allrunsHS['Sample'].str.startswith('C')]



In [ ]:
sns.scatterplot(x='Sample', y='FOLD_80_BASE_PENALTY', hue='Run', data=CNVsamples)
plt.axhline(y=1.3,color='r',linestyle='--')
plt.figszie=(500,100)
plt.xticks(rotation=90)

In [ ]:
CNVsamples.sort_values(by=['FOLD_80_BASE_PENALTY'],ascending=False)[['Sample'  ,'FOLD_80_BASE_PENALTY','Run']]

In [ ]:
import dxpy

In [ ]:
project_ids = []
projects=dxpy.bindings.search.find_projects(name="002_*A*_CEN",name_mode="glob",describe=True)
for project in projects:
    project_ids.append(project['describe']['id'])

In [ ]:
print(project_ids)

In [ ]:
example_id='project-GBkKPvQ4kYyF88304g4fzbYv'
example_id2='project-GPy5K684zvb1GXyBb1B2gxf6'
project_name=dxpy.api.project_describe(example_id)['name'][4:33]

hs_metrics = list(dxpy.bindings.search.find_data_objects(classname="file",describe=True, project=example_id2,name="multiqc_picard_HsMetrics.txt",name_mode="exact"))
live_files=[x for x in hs_metrics if x['describe']['archivalState'] == 'live']

if live_files:
    print(live_files)
else:
    print('No hs_metrics found')


In [ ]:

for project in project_ids:
    project_name=dxpy.api.project_describe(project)['name'][4:33]
    #print(project_name)
    hs_metrics = list(dxpy.bindings.search.find_data_objects(classname="file",describe=True, project=project,name="multiqc_picard_HsMetrics.txt",name_mode="exact"))
    live_files=[x for x in hs_metrics if x['describe']['archivalState'] == 'live']

    if live_files:
        dxpy.bindings.dxfile_functions.download_dxfile(live_files[0]['id'],filename=f'{project_name}-hs-metrics.txt',project=project)

In [ ]:
import os
data=pd.DataFrame()
for filename in os.listdir(PATH):
    if filename.endswith("hs-metrics.txt"):
        run_name=filename[:-15]
        tmp_df=pd.read_csv(f'{PATH}/{filename}', sep='\t')
        tmp_df['Run']=run_name
        data = pd.concat([data,tmp_df])

In [ ]:
data.Run.unique()

In [ ]:
plt.style.use('ggplot')
sns.histplot(data=data.FOLD_80_BASE_PENALTY,bins=40)

data.shape


In [ ]:
plt.style.use('ggplot')
above_cutoff=data.loc[data['FOLD_80_BASE_PENALTY']>1.3]
sns.histplot(data=above_cutoff.FOLD_80_BASE_PENALTY,bins=15)
#plt.ayhline(x=1.3,color='r',linestyle='--')

In [10]:
print(len(list(data.Run.unique())))
above_cutoff_high=data.loc[data['FOLD_80_BASE_PENALTY']>1.5]

print(len(list(above_cutoff_high.Run.unique())))
print(len(list(above_cutoff.Run.unique())))

In [ ]:
sns.histplot(data=above_cutoff_high.FOLD_80_BASE_PENALTY,bins=20)


In [ ]:
ax=sns.scatterplot(x='Sample', y='FOLD_80_BASE_PENALTY', hue='Run', data=above_cutoff_high)
sns.move_legend(ax, 'upper left',bbox_to_anchor=(1,1))
labels=plt.xticks(rotation=90)

In [ ]:
sns.countplot(x='Run', data=above_cutoff_high,order=above_cutoff_high['Run'].value_counts().index)
x=plt.xticks(rotation=90)

In [ ]:
above_cutoff_high[above_cutoff_high.Sample.str.startswith('X224279')].FOLD_80_BASE_PENALTY


## Investigate in conjuction with gcnv.bed

Downloaded and unziped following file for run 124 file-GP6X5484GxyxP0JY5V9j2pY5


In [ ]:
gcnv_124=pd.read_csv(f'{PATH}/002_221130_A01303_0124_AHMTC5DRX2_CEN_copy_ratios-chr20.gcnv.bed', sep='\t')
chr20_gcnv_mean = gcnv_124.mean().drop(['chr','start','end']).to_frame(name='CP_mean')
chr20_gcnv_mean

In [ ]:
subset124=data[data.Run=='221130_A01303_0124_AHMTC5DRX2'][['Sample','FOLD_80_BASE_PENALTY', 'HS_PENALTY_20X', 'HS_PENALTY_30X', 'HS_PENALTY_40X', 'HS_PENALTY_50X',
       'HS_PENALTY_100X']].sort_values(by=['FOLD_80_BASE_PENALTY'],ascending=False).set_index('Sample')

In [ ]:
combined = chr20_gcnv_mean.join(subset124, how='inner')

In [ ]:
combined.sort_values(by=['FOLD_80_BASE_PENALTY'])

In [ ]:
sns.scatterplot(data=combined,x=combined.CP_mean,y=combined.FOLD_80_BASE_PENALTY)

In [ ]:
fig,axs = plt.subplots(2,3)
ax1 = sns.scatterplot(data=combined,x=combined.CP_mean,y=combined.HS_PENALTY_30X)